# Social Media Monitoring

## Organization List


The following table contains data on the organizations and their associated social acounts

In [ ]:
import qgrid
from smm_wrapper import SMM
from wikiwho_wrapper import WikiWho
qgrid.set_grid_option('maxVisibleRows', 5)

# Create an instance to the smm wrapper
smm = SMM(unit = "organizations")

# Request the politicians from the API
df = smm.dv.get_organizations()
df = df[(df['category']=='political')]
# Show the result
qgrid.show_grid(df)

## Instructions

Please choose an `subcategory`, which indicates a political party, from the table above and enter its name in the field below:

In [ ]:
party = 'afd'
aggregation = 'month' #aggregation can be by day ('day'), week ('week') or month('month')

In [ ]:
import pandas as pd

# Filter the accounts by party, and valid ones (the ones that contain wp_ids)
party_df = df[(df['subcategory']==party) & (df['wp_ids'].notnull())] 

# query the Social Media Monitoring API
wiki_chobs = pd.concat(smm.dv.wikipedia(wikipedia_page_id=wp_id, aggregate_by=aggregation) 
               for wp_ids in party_df['wp_ids'] for wp_id in wp_ids) 

#using wikiwho to extract conflicts and revisions
ww = WikiWho(lng='de')
wiki_data = pd.concat(ww.dv.edit_persistence(page_id=wp_id) for wp_ids in party_df['wp_ids'] for wp_id in wp_ids )
wiki_data['undos'] = wiki_data['dels'] + wiki_data['reins']
wiki_data = wiki_data[['year_month','conflicts','revisions', 'undos']]

# aggregate the wikipedia chobs
total_wiki_chobs = wiki_chobs.groupby('date').agg({'chobs': 'sum'}).reset_index()

In [ ]:
#aggregating the data from wikiwho and grouping it together with chobs
wiki_data = wiki_data.groupby('year_month').agg(
    {
        'conflicts': 'sum',
        'revisions': 'sum',
        'undos': 'sum'
    }).reset_index()
wiki_total = pd.merge(total_wiki_chobs, wiki_data, left_on=total_wiki_chobs['date'], right_on=pd.to_datetime(wiki_data['year_month'])).reset_index()

## Plotting

In [ ]:
import plotly
from plotly import graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

plotly.offline.iplot({
    "data": [go.Scatter(x=wiki_total['date'], y=wiki_total['chobs'], name='Change objects'),
            go.Scatter(x=wiki_total['date'], y=wiki_total['revisions'], name='Revisions')], 
    "layout": go.Layout(hovermode='closest', title='Wikipedia (chobs and revisions)', xaxis={'title':'Date'}, yaxis={'title':'N'}, showlegend=True)
})

plotly.offline.iplot({
    "data": [go.Scatter(x=wiki_total['date'], y=wiki_total['undos'], name='Undos'),
            go.Scatter(x=wiki_total['date'], y=wiki_total['conflicts'], name='Conflicts')], 
    "layout": go.Layout(hovermode='closest', title='Wikipedia (undos and conflicts)', xaxis={'title':'Date'}, yaxis={'title':'N'}, showlegend=True)
})